In [1]:
from math import atan, sin, cos, pi, sqrt
from scipy.constants import g
import numpy as np
from scipy.integrate import RK45

In [2]:
m = 1    # Mass

b = 0.3  # Damping coefficient
h = 0.5  # Height above the x-y plane

L = g  # bob length
R = 0.5  # magnet radius

rmag = sqrt((L+h)**2 + R**2)
θmag = atan(R/(L+h))
ϕmag_list = np.array([(pi/2) * i for i in range(4)])  # -> magnets at (0, R), (0, -R), (-R, 0), (R, 0)
strength = 1
polarity_list = np.array([1,1,1,1]) * strength

## define eqns

In [3]:
def polar_distance(r1_v, r2_v):
    (r1,t1,p1) = r1_v
    (r2,t2,p2) = r2_v
    return sqrt(
                (r1)**2 + (r2)**2
                - 2*(r1)*(r2)*(sin(t1)*sin(t2)*cos(p1-p2) + cos(t1)*cos(t2))
            )

In [7]:
def force_by_magnets(θ, ϕ, θv, ϕv):
    """this is so ugly"""
    distance = lambda ϕmag: polar_distance((L,θ,ϕ), (rmag,θmag,ϕmag))
    
    def force_by_single_magnet_θ(polarity, ϕmag):
        direction_component = cos(θ)*sin(θmag)*cos(ϕ-ϕmag) - sin(θ)*cos(θmag)
        return (polarity/(distance(ϕmag))**5) * direction_component
    
    def force_by_single_magnet_ϕ(polarity, ϕmag):
        direction_component = -sin(θmag)*sin(ϕ-ϕmag)        
        return  (polarity/(distance(ϕmag))**5) * direction_component

    return np.array([
        sum(np.vectorize(force_by_single_magnet_θ)(polarity_list, ϕmag_list)),
        sum(np.vectorize(force_by_single_magnet_ϕ)(polarity_list, ϕmag_list))
    ])

def force_by_gravity(θ, ϕ, θv, ϕv):
    return np.array([
        -m*g*sin(θ),
        0
    ])

def force_of_damping(θ, ϕ, θv, ϕv):
    return -b*L * np.array([
        θv,
        ϕv*sin(θ)
    ])

In [13]:
def ddt(t, y_array):
    θ, ϕ, θv, ϕv = y_array
    force_θ, force_ϕ = (force_by_gravity(θ, ϕ, θv, ϕv)
                        + force_by_magnets(θ, ϕ, θv, ϕv)
                        + force_of_damping(θ, ϕ, θv, ϕv))
    return np.array([
        θv,
        ϕv,
        (force_θ + L*(ϕv**2)*sin(θ)*cos(θ))/L,
        (force_ϕ - 2*L*θv*ϕv*cos(θ))/(L*sin(θ))
    ])

## run pendlum model

In [36]:
starting_conditions = np.array([
    pi/6,
    pi/6,
    0,
    0
])

integrator = RK45(ddt, 0, starting_conditions, t_bound=100, first_step=0.001, max_step=0.01)

In [25]:
ddt(0, starting_conditions)

array([ 0.00000000e+00,  0.00000000e+00, -5.00053651e-01, -4.18021025e-08])

In [39]:
result = []
while not (integrator.status == 'finished'):
    path.append([integrator.t, *integrator.y])
    integrator.step()
    

In [56]:
result = np.array(path)
to_cartesian = lambda result_v: ########################################## 

## graphing

In [57]:
from matplotlib import projections
import matplotlib.pyplot as plt
import numpy as np

import matplotlib.animation as animation

def plot_trajectory(trajectory, magnets):
    
    ax = plt.figure(figsize=(8, 8)).add_subplot(projection='3d')
    # plt.title(f'Magnetic Pendulum Trajectory')
    ax.set_title(f'Magnetic Pendulum Trajectory')
    # Plot the trajectory
    ax.plot(*trajectory.T, label='Pendulum Path', zorder=1)  

    # Plot the magnet positions
    ax.scatter(*zip(*[m.pos.to_cartesian().coords.astype(np.double) for m in magnets]), color='red', label='Magnets', zorder=2)  

    # Plot the final & initial positions of the pendulum
    final_position = trajectory[-1]
    initial_position = trajectory[0]
    ax.scatter(*final_position, color='cyan', label='Final Position', s=10, zorder=3)
    ax.scatter(*initial_position, color='black', label='Initial Position', s=10, zorder=3)

    ax.legend()
    ax.grid(True)
    ax.axis('equal')
    plt.show()

def plot_trajectory_rotate(trajectory, magnets):

    ax = plt.figure(figsize=(8, 8)).add_subplot(projection='3d')
    # plt.title(f'Magnetic Pendulum Trajectory')
    ax.set_title(f'Magnetic Pendulum Trajectory')
    # Plot the trajectory
    ax.plot(*trajectory.T, label='Pendulum Path', zorder=1)  

    # Plot the magnet positions
    ax.scatter(*zip(*[m.pos.to_cartesian().coords.astype(np.double) for m in magnets]), color='red', label='Magnets', zorder=2)  

    # Plot the final & initial positions of the pendulum
    final_position = trajectory[-1]
    initial_position = trajectory[0]
    ax.scatter(*final_position, color='cyan', label='Final Position', s=10, zorder=3)
    ax.scatter(*initial_position, color='black', label='Initial Position', s=10, zorder=3)

    ax.legend()
    ax.grid(True)
    ax.axis('equal')


    # Rotate the axes and update
    while True:
        for angle in range(0, 360):
            # Normalize the angle to the range [-180, 180] for display
            angle_norm = (angle + 180) % 360 - 180
            # Cycle through a full rotation of elevation, then azimuth, roll, and all
            elev = azim = roll = 0
            elev = 13
            azim = angle_norm


            # Update the axis view and title
            ax.view_init(elev, azim, roll)
            plt.title('Elevation: %d°, Azimuth: %d°, Roll: %d°' % (elev, azim, roll))

            plt.draw()
            plt.pause(.001)

def plot_trajectory_animation(trajectory, magnets, dt=0.01):

    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(projection="3d")
    # plt.title(f'Magnetic Pendulum Trajectory')
    ax.set_title(f'Magnetic Pendulum Trajectory')
    # Plot the trajectory
    ax.plot(*trajectory.T, label='Pendulum Path', color='gray', alpha=0.1, zorder=1)  

    # Plot the magnet positions
    ax.scatter(*zip(*[m.pos.to_cartesian().coords.astype(np.double) for m in magnets]), color='red', label='Magnets', zorder=2)  

    # Plot the final & initial positions of the pendulum
    final_position = trajectory[-1]
    initial_position = trajectory[0]
    ax.scatter(*final_position, color='cyan', label='Final Position', s=10, zorder=3)
    ax.scatter(*initial_position, color='black', label='Initial Position', s=10, zorder=3)

    ax.legend()
    ax.grid(True)
    ax.axis('equal') 

    line, = ax.plot([], [], [], 'o-', lw=2)
    trace, = ax.plot([], [], [], '.-', lw=1, ms=2)
    time_template = 'time = %.1f'
    time_text = ax.text(0.05, 0.9, 0.9, '', transform=ax.transAxes)


    def animate(i):
        this = trajectory[i:i+1]
        history = trajectory[:i]

        line.set_data_3d(this.T)
        trace.set_data_3d(history.T)
        time_text.set_text(time_template % (i))
        return line, trace, time_text

    N = len(trajectory)
    ani = animation.FuncAnimation(
        fig, animate, N, interval=dt/3, blit=True)
    plt.show()